# Global Optimizers tests

## Scipy

#### Dual Annealing

In [3]:
from scipy.optimize import dual_annealing
import numpy as np

func = lambda x: np.sum(x*x - 10*np.cos(2*np.pi*x)) + 10*np.size(x)
lw = [-5.12] * 10
up = [5.12] * 10

In [4]:
ret = dual_annealing(func, bounds=list(zip(lw, up)), seed=1234)
print("global minimum: xmin = {0}, f(xmin) = {1:.6f}".format(ret.x, ret.fun))

global minimum: xmin = [-4.26437714e-09 -3.91699361e-09 -1.86149218e-09 -3.97165720e-09
 -6.29151648e-09 -6.53145322e-09 -3.93616815e-09 -6.55623025e-09
 -6.05775280e-09 -5.00668935e-09], f(xmin) = 0.000000


#### Simplicial Homology Global Optimization (SHGO)

In [6]:
from scipy.optimize import rosen, shgo
bounds = [(0,2), (0, 2), (0, 2), (0, 2), (0, 2)]

Bounded:

In [7]:
result = shgo(rosen, bounds)
result.x, result.fun

(array([1., 1., 1., 1., 1.]), 2.920392374190081e-18)

Empty bounds:

In [8]:
bounds = [(None, None), ]*4
result = shgo(rosen, bounds)
result.x

array([0.99999851, 0.99999704, 0.99999411, 0.9999882 ])

Egg holder function (many local minima, one global minimum):

In [9]:
def eggholder(x):
    return (-(x[1] + 47.0)
            * np.sin(np.sqrt(abs(x[0]/2.0 + (x[1] + 47.0))))
            - x[0] * np.sin(np.sqrt(abs(x[0] - (x[1] + 47.0))))
            )
bounds = [(-512, 512), (-512, 512)]

In [12]:
result = shgo(eggholder, bounds, n=30, sampling_method='sobol')
result.xl, result.funl

(array([[ 512.        ,  404.23180542],
        [ 283.07593402, -487.12566542],
        [-294.66820039, -462.01964031],
        [-105.87688985,  423.15324143],
        [-242.97923629,  274.38032063],
        [-506.25823477,    6.3131022 ],
        [-408.71980114, -156.10116115],
        [ 150.23210485,  301.31378508],
        [  91.00921872, -391.28375655],
        [ 202.8966344 , -269.38042147],
        [ 361.66625957, -106.96490692],
        [-219.40615102, -244.06022436],
        [ 151.59603137, -100.61082677]]),
 array([-959.64066272, -718.16745962, -704.80659592, -565.99778097,
        -559.78685655, -557.36868733, -507.87385942, -493.9605115 ,
        -426.48799655, -421.15571437, -419.31194957, -410.98477763,
        -202.53912972]))

In [13]:
result_2 = shgo(eggholder, bounds, n=60, iters=5, sampling_method='sobol')
len(result.xl), len(result_2.xl)

(13, 39)

#### Basin Hopping

In [14]:
from scipy.optimize import basinhopping
func = lambda x: np.cos(14.5 * x - 0.3) + (x + 0.2) * x
x0=[1.]

In [15]:
minimizer_kwargs = {"method": "BFGS"}
ret = basinhopping(func, x0, minimizer_kwargs=minimizer_kwargs, niter=200)
print("global minimum: x = %.4f, f(x0) = %.4f" % (ret.x, ret.fun))

global minimum: x = -0.1951, f(x0) = -1.0009


2D minimization problem:

In [16]:
def func2d(x):
    f = np.cos(14.5 * x[0] - 0.3) + (x[1] + 0.2) * x[1] + (x[0] + 0.2) * x[0]
    df = np.zeros(2)
    df[0] = -14.5 * np.sin(14.5 * x[0] - 0.3) + 2. * x[0] + 0.2
    df[1] = 2. * x[1] + 0.2
    return f, df

In [17]:
minimizer_kwargs = {"method":"L-BFGS-B", "jac":True}
x0 = [1.0, 1.0]
ret = basinhopping(func2d, x0, minimizer_kwargs=minimizer_kwargs, niter=200)
print("global minimum: x = [%.4f, %.4f], f(x0) = %.4f" % (ret.x[0],
                                                          ret.x[1],
                                                          ret.fun))

global minimum: x = [-0.1951, -0.1000], f(x0) = -1.0109


Prints output at every step:

In [18]:
def print_fun(x, f, accepted):
        print("at minimum %.4f accepted %d" % (f, int(accepted)))

In [19]:
np.random.seed(1)
ret = basinhopping(func2d, x0, minimizer_kwargs=minimizer_kwargs,
                   niter=10, callback=print_fun)

at minimum 0.4159 accepted 1
at minimum -0.9073 accepted 1
at minimum -0.1021 accepted 1
at minimum -0.1021 accepted 1
at minimum 0.9102 accepted 1
at minimum 0.9102 accepted 1
at minimum 2.2945 accepted 0
at minimum -0.1021 accepted 1
at minimum -1.0109 accepted 1
at minimum -1.0109 accepted 1


Bounded search:

In [21]:
class MyBounds(object):
    def __init__(self, xmax=[1.1,1.1], xmin=[-1.1,-1.1] ):
        self.xmax = np.array(xmax)
        self.xmin = np.array(xmin)
    def __call__(self, **kwargs):
        x = kwargs["x_new"]
        tmax = bool(np.all(x <= self.xmax))
        tmin = bool(np.all(x >= self.xmin))
        return tmax and tmin

In [22]:
mybounds = MyBounds()
ret = basinhopping(func2d, x0, minimizer_kwargs=minimizer_kwargs, niter=10, accept_test=mybounds)

## DEAP - Distributed Evolutionary Algorithms in Python 

In [23]:
def ackley(x):
    """
    Ackley function, 2 dimensional.
    :param x: List of parameters.
    :return: Function result, using the given x parameters.
    """
    arg1 = -0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * x[0]) + np.cos(2. * np.pi * x[1]))
    return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e

In [74]:
import random
from deap import algorithms, base, creator, tools
import numpy as np

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# creator.create("Individual", list, fitness=creator.FitnessMax)
creator.create("Individual", list, fitness=creator.FitnessMin)

def evalOneMax(individual):
    x = ackley(individual)
    return_value = 1 /  np.sum(x)
    return (return_value,)

# def evalOneMax(individual):
#     return ackley(individual)

toolbox = base.Toolbox()
# toolbox.register("attr_bool", random.randint, 0, 1)
# toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=2)
toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

if __name__ == "__main__":
    pop = toolbox.population(n=10)
    
    # One liner black box:
    # algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, verbose=False) # OR
    
    # TODO: find all EA variations -> ~10 GA island versions
    
    # TODO: Gray/white box:    
    ngen, cxpb, mutpb = 20, 0.5, 0.2
    fitnesses = toolbox.map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(ngen):
        # TODO: island communication IO
        # TODO: individual attribs should be float in [0, 1]
        # TODO: rescale-encapsulate to NAS Ozone DNN space        
        pop = toolbox.select(pop, k=len(pop))
        pop = algorithms.varAnd(pop, toolbox, cxpb, mutpb)
        print("len(pop)", len(pop))  # TODO: Island swap specific individuals
        
        print("pop[0]", pop[0])
        if g == 3:
            print("-- Changing in generation 12 an individual")
            pop[0][0] = 0.5
            pop[0][1] = 0.5
            print("-- pop[0]", pop[0])            
            print("-- ", type(pop[0]))
            
        
        invalids = [ind for ind in pop if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalids)
        for ind, fit in zip(invalids, fitnesses):
            ind.fitness.values = fit
    
    print(tools.selBest(pop, k=1))

len(pop) 10
pop[0] [0.9886053648611993, 0.28563026235425826]
len(pop) 10
pop[0] [0.9555129406253036, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
-- Changing in generation 12 an individual
-- pop[0] [0.5, 0.5]
--  <class 'deap.creator.Individual'>
len(pop) 10
pop[0] [0.7102169876096865, 0.5]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
pop[0] [0.7102169876096865, 0.6823823944450375]
len(pop) 10
